# Учебный проект 17_Разработка системы компьютерного зрения для определения возраста покупателей

## Содержание

* [Описание проекта](#Описание)
* [Импорт библиотек Python](#Импорт)
* [Загрузка данных](#Загрузка)
* [Построение модели машинного обучения](#Моделирование)
* [Общий вывод](#Вывод)

## Описание проекта <a class = 'anchor' id = 'Описание'></a>

На исследовании находятся данные с `фотографиями покупателей и их возрасте`, которые были предоставлены руководством компании "Хлеб-Соль".

---

`Задача`

Разработать модель машинного обучения для **вычисления приблизительного возраста покупателей** с целью:
* анализа покупок и предложения товаров для возрастных групп;
* контроля добросовестности кассиров при продаже алкоголя.

Необходимо добиться максимально возможной **низкой MAE**.

---

`Описание данных`

* папка `/datasets/faces/` с фотографиями;
* файл `CSV` с разметкой, содержащий два столбца:
    * `file_name` — название файла фотографии;
    * `real_age` — возраст человека на фотографии

---

`Путь решения`

1. Собрать исторические данные о покупателях магазина;
2. Провести предобработку значений в наборе данных;
3. Провести исследовательский анализ для проектирования модели:
    * Изучить размер выборки;
    * Построить график распределения возраста в выборке;
    * Вывести на экран 10-15 фотографий покупателей и изучить структуру датасета.
4. Построить модель машинного обучения для вычисления приблизительного возраста покупателей с перебором различных параметров и конфигураций;
5. Сформировать вывод о подготовленных решениях.

## Импорт библиотек Python <a class = 'anchor' id = 'Импорт'></a>

Данный блок характеризуется следующими последовательными действиями:
1. Импорт библиотек Python:
    * для манипулирования данными;
    * для визуализации данных;
    * для решения задач машинного обучения:
        * инструменты проектирования нейронных сетей для анализа изображений;
        * метрики оценки эффективности моделей;
        * механизмы отключения предупреждений.
2. Инициализация переменных-констант для последующего использования на этапе построения моделей МО;
3. Формирование вывода по итогам данного этапа.

In [1]:
%pip install tensorflow --upgrade


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# импорт библиотек python

# для манипулирования данными
import pandas as pd
import numpy as np

# для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

# инструменты проектирования нейронных сетей для анализа изображений
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet50

# игнорирование возможных предупреждений
import warnings
warnings.filterwarnings("ignore")

In [3]:
# инициализация констант для дальнейшего использования в проекте
# инициализация переменной SEED для фиксирования случайности
# инициализация переменной TEST_SIZE для фиксирования размера тестовой выборки при разбиении наборов данных
SEED = 12345
TEST_SIZE = 0.25

**Вывод**

1. Импортированы библиотеки Python:
    * для манипулирования данными:
        * pandas;
        * numpy.
    * для визуализации данных:
        * matplotlib.pyplot;
        * seaborn.
    * для решения задач машинного обучения:
        * инструменты проектирования нейронных сетей для анализа изображений;
        * метрики оценки эффективностей модели:
            * mean_absolute_error - средняя абсолютная ошибка.
    * для отключения предупреждений.
2. Инициализированы переменные:
    * **TEST_SIZE** для фиксирования размера тестовой выборки при разбиении наборов данных;
    * **SEED** для фиксирования случайности.

## Загрузка данных <a class = 'anchor' id = 'Загрузка'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Загрузка данных в рабочую среду Jupyter Notebook - инициализация переменной `datagen` для загрузки данных из директории с изображениями;
2. Вывод на экран размерности выборок:
    * тренировочной - инициализация переменной datagen_flow_train;
    * валидационной - инициализация переменной datagen_flow_test.
3. Загрузка данных из файла `CSV` с разметкой;
4. Вывод на экран общей информации о наборе данных:
    * вывод общей структуры набора данных - демонстрация первых 5 строк;
    * общей информации о наборе данных;
    * распределение количественных величин в наборе данных.
5. Формирование вывода по итогам данного этапа.

In [ ]:
# загрузка данных в рабочую среду Jupyter Notebook

# инициализация переменной datagen для загрузки данных из директории с изображениями
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=TEST_SIZE,
                             horizontal_flip = True,
                             vertical_flip = True,
                             rotation_range=90)

# инициализация переменной datagen_flow_train
datagen_flow_train = datagen.flow_from_directory(
    '/datasets/faces/',
    target_size=(150, 150),
    batch_size=16,
    class_mode='sparse',
    subset='training',
    seed=SEED)

# инициализация переменной datagen_flow_test
datagen_flow_test = datagen.flow_from_directory(
    '/datasets/faces/',
    target_size=(150, 150),
    batch_size=16,
    class_mode='sparse',
    subset='validation',
    seed = SEED)

In [ ]:
# загрузка данных из файла CSV с разметкой
labels = pd.read_csv('/datasets/faces/labels.csv')

# инициализация пользовательской функции для первичного изучения содержимого наборов данных
def first_meeting (df : pd.DataFrame, df_name : str) -> None:
    print(f'Структура набора данных {df_name}')
    display(df.head())
    print('Общая информация о наборе')
    print(df.info())
    print()

In [ ]:
# вывод на экран структуры и основных параметров датасета
first_meeting(labels, 'labels')

In [ ]:
# инициализация пользовательской функции построения распределений количественных непрерывных показателей
def num_distribution(df : pd.DataFrame, column : str, bins : int):
    plt.subplot(1, 2, 1)
    plt.xlabel(f'Значения признака {column}')
    plt.ylabel(f'Частота значений признака')
    plt.title(f'Гистограмма значений {column}', fontsize = 10)
    sns.histplot(data = df, x = df[column], bins = bins)
    plt.subplot(1, 2, 2)
    plt.xlabel(f'Значения признака {column}')
    plt.title(f'Диаграмма размаха значений {column}', fontsize = 10)
    sns.boxplot(data = df, x = df[column])
    plt.grid(False)
    plt.show()

In [ ]:
# инициализация пользовательской функции по построению гистограмм по передаваемым метрикам
def histogram_plotting(data: pd.DataFrame, feature : str, bins: int, x_size: int, y_size: int, feature_xlabel : str):
    # вычисление статистических метрик для дальнейшей визуализации
    q1 = data[feature].quantile(0.25)
    q3 = data[feature].quantile(0.75)
    upper_bound = q3 + 1.5 * (q3 - q1)
    lower_bound = q1 - 1.5 * (q3 - q1)

    # построение визуализации
    plt.figure(figsize = (x_size, y_size))
    plt.hist(data[feature], color = 'blue', edgecolor = 'white', bins = bins)
    plt.axvline(upper_bound, c = 'red', ls = '-', label = 'верхняя граница допустимых значений')
    plt.axvline(q3, c = 'red', ls = '--', label = '3 квартиль значений')
    plt.axvline(q1, c = 'black', ls = '--', label = '1 квартиль значений')
    plt.axvline(lower_bound, c = 'black', ls = '-', label = 'нижняя граница допустимых значений')
    plt.title(f'Гистограмма распределения значений по метрике: {feature_xlabel}', fontsize = 10)
    plt.xlabel(feature_xlabel)
    plt.ylabel('Количество значений по метрике')
    plt.legend(bbox_to_anchor = (1, 0.6))
    plt.show()

    # вывод статистических метрик на экран
    print('Верхняя допустимая граница значений:', upper_bound)
    print('Нижняя допустимая граница значений:', lower_bound)
    print('Медианное значение:', data[feature].median())
    print('Среднее значение:', round(data[feature].mean(), 2))

    # расчет доли аномальных значений по метрике
    print('Доля значений, выходящих за верхнюю границу: {:.2%}'.format(data[data[feature] > upper_bound].shape[0] / data[feature].shape[0]))
    print('Доля значений, выходящих за нижнюю границу: {:.2%}'.format(data[data[feature] < lower_bound].shape[0] / data[feature].shape[0]))

In [ ]:
# извлечение батча данных и разбиение на признаки и классы изображений
features, target = next(datagen_flow_train)

# выводим 16 изображений
fig = plt.figure(figsize=(10,10))
for i in range(16):
    fig.add_subplot(4, 4, i+1)
    plt.imshow(features[i])
    # для компактности удаляем оси и прижимаем изображения друг к другу
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

**Вывод**

1. Произведена загрузка данных в рабочую среду Jupyter Notebook. Инициализирована переменная `datagen`. Набор данных состоит из 7_591 изображений;
2. Произведено разделение исходного набора на выборки:
    * `datagen_flow_train` - обучающая выборка;
    * `datagen_flow_test` - тестовая выборка.
3. Выведены на экран основные параметры набора данных `labels`:
    * набор данных состоит из 2 столбцов и 7591 строк;
    * в наборе отсутствуют пропущенные значения.
4. Выведены на экран основные параметры распределения данных по признаку `real_age`:
    * распределение имеет вид, приближенный к нормальному, с небольшим скосом вправо. Стоит отметить, что в наборе присутствуют так же слишком юные покупатели - дети;
    * присутствуют выбросы в виде аномально больших значений - их доля составляет 2.23% от всего набора;
    * медианное значение возраста покупателей - 29 лет;
    * средний возраст покупателей - 31.2 лет.

В целом, набор данных не требует дополнительных преобразований помимо тех, что уже были выполнены на этапе загрузки.
При построении модели нейронной сети и при дальнейшем получении прогнозов будем учитывать, что в наборе присутствуют аномальные значения. А значит, итоговые прогнозные значения могут быть выше, чем реальный возраст покупателей.

## Построение модели машинного обучения <a class = 'anchor' id = 'Моделирование'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Построение модели нейронной сети с инициализацией пользовательских функций:
    * load_train - загрузка обучающего набора данных;
    * load_test - загрузка тестового набора данных;
    * create_model - создание модели нейронной сети;
    * train_model - обучение модели нейронной сети.
2. Формирование вывода по итогам данного этапа.

**Код, приведенный ниже, прошел предварительное исполнение на отдельном GPU-тренажере**

In [ ]:
# функция загрузки обучающей выборки
def load_train(path):
    labels = pd.read_csv("/datasets/faces/labels.csv")
    train_datagen = ImageDataGenerator(
        vertical_flip=True,
        rotation_range=90,
        width_shift_range=0.3,
        height_shift_range=0.3,
        rescale=1.0 / 255,
        validation_split=TEST_SIZE,
    )

    train_datagen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory="/datasets/faces/final_files/",
        x_col="file_name",
        y_col="real_age",
        target_size=(224, 224),
        batch_size=32,
        class_mode="raw",
        subset="training",
        seed=SEED,
    )

    return train_datagen_flow


# функция загрузки валидационной выборки
def load_test(path):
    labels = pd.read_csv("/datasets/faces/labels.csv")
    test_datagen = ImageDataGenerator(
        validation_split=TEST_SIZE,
        rescale=1.0 / 255)

    test_datagen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory="/datasets/faces/final_files/",
        x_col="file_name",
        y_col="real_age",
        target_size=(224, 224),
        batch_size=32,
        class_mode="raw",
        subset="validation",
        seed=SEED,
    )

    return test_datagen_flow


# функция создания модели нейронной сети
def create_model(input_shape):
    # импортируем архитектуру ResNet
    backbone = ResNet50(
        input_shape=input_shape,
        weights="/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5",
        include_top=False,
    )
    # инициализация модели
    model = Sequential()
    # добавление слоёв в модель
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation="relu"))
    # подготовка модели к обучению
    optimizer = Adam(learning_rate = 0.00005)
    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_absolute_error"]
    )
    # результат: возвращение настроенной модели
    return model


# функция обучения модели
def train_model(
    model,
    train_datagen_flow,
    test_datagen_flow,
    batch_size=None,
    epochs=20,
    steps_per_epoch=None,
    validation_steps=None,
):

    model.fit(
        train_datagen_flow,
        validation_data=test_datagen_flow,
        batch_size=batch_size,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=2,
        shuffle=True,
    )

    return model

**Скопирован результат исполнения кода в отдельном GPU-тренажере**


```Train for 178 steps, validate for 60 steps
Epoch 1/20
178/178 - 99s - loss: 399.3076 - mean_absolute_error: 14.9709 - val_loss: 615.4288 - val_mean_absolute_error: 19.8011
Epoch 2/20
178/178 - 92s - loss: 145.7714 - mean_absolute_error: 9.2002 - val_loss: 804.7238 - val_mean_absolute_error: 23.4426
Epoch 3/20
178/178 - 92s - loss: 126.4676 - mean_absolute_error: 8.5357 - val_loss: 403.6989 - val_mean_absolute_error: 15.0621
Epoch 4/20
178/178 - 92s - loss: 113.5236 - mean_absolute_error: 8.0955 - val_loss: 178.7277 - val_mean_absolute_error: 10.0013
Epoch 5/20
178/178 - 92s - loss: 102.9207 - mean_absolute_error: 7.7567 - val_loss: 147.3729 - val_mean_absolute_error: 9.0171
Epoch 6/20
178/178 - 92s - loss: 95.1175 - mean_absolute_error: 7.4542 - val_loss: 96.8540 - val_mean_absolute_error: 7.4767
Epoch 7/20
178/178 - 92s - loss: 88.7458 - mean_absolute_error: 7.1451 - val_loss: 92.6042 - val_mean_absolute_error: 7.2815
Epoch 8/20
178/178 - 92s - loss: 82.3103 - mean_absolute_error: 6.9123 - val_loss: 92.7663 - val_mean_absolute_error: 7.3931
Epoch 9/20
178/178 - 93s - loss: 78.4509 - mean_absolute_error: 6.7620 - val_loss: 104.8380 - val_mean_absolute_error: 7.7020
Epoch 10/20
178/178 - 92s - loss: 73.2345 - mean_absolute_error: 6.5186 - val_loss: 99.7994 - val_mean_absolute_error: 7.3209
Epoch 11/20
178/178 - 92s - loss: 67.0615 - mean_absolute_error: 6.3234 - val_loss: 108.0002 - val_mean_absolute_error: 7.6862
Epoch 12/20
178/178 - 92s - loss: 64.7014 - mean_absolute_error: 6.1460 - val_loss: 110.9409 - val_mean_absolute_error: 7.7313
Epoch 13/20
178/178 - 92s - loss: 59.8807 - mean_absolute_error: 5.9705 - val_loss: 94.1817 - val_mean_absolute_error: 7.3128
Epoch 14/20
178/178 - 92s - loss: 59.2435 - mean_absolute_error: 5.9159 - val_loss: 94.7431 - val_mean_absolute_error: 7.3108
Epoch 15/20
178/178 - 92s - loss: 54.1986 - mean_absolute_error: 5.6681 - val_loss: 84.1356 - val_mean_absolute_error: 7.0009
Epoch 16/20
178/178 - 92s - loss: 52.8570 - mean_absolute_error: 5.5886 - val_loss: 91.8597 - val_mean_absolute_error: 7.1832
Epoch 17/20
178/178 - 92s - loss: 50.1566 - mean_absolute_error: 5.4451 - val_loss: 94.5077 - val_mean_absolute_error: 7.3152
Epoch 18/20
178/178 - 92s - loss: 48.3286 - mean_absolute_error: 5.4123 - val_loss: 105.5588 - val_mean_absolute_error: 7.5545
Epoch 19/20
178/178 - 92s - loss: 47.1229 - mean_absolute_error: 5.2773 - val_loss: 87.3449 - val_mean_absolute_error: 6.9815
Epoch 20/20
178/178 - 92s - loss: 44.8868 - mean_absolute_error: 5.1817 - val_loss: 92.2197 - val_mean_absolute_error: 7.2710
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
60/60 - 9s - loss: 92.2197 - mean_absolute_error: 7.2710
Test MAE: 7.2710
```

**Вывод**

1. Построена модель нейронной сети для анализа изображений покупателей. Инициализированы пользовательские функции:
    * load_train - загрузка обучающего набора данных;
    * load_test - загрузка тестового набора данных;
    * create_model - создание модели нейронной сети;
    * train_model - обучение модели нейронной сети.
2. По результатам обучения модели на GPU-тренажере, удалось достичь целевого значения метрики `MAE` - 7.2710.

## Общий вывод <a class = 'anchor' id = 'Вывод'></a>

1. Импортированы библиотеки Python:
    * для манипулирования данными:
        * pandas;
        * numpy.
    * для визуализации данных:
        * matplotlib.pyplot;
        * seaborn.
    * для решения задач машинного обучения:
        * инструменты проектирования нейронных сетей для анализа изображений.
    * для отключения предупреждений.
2. Инициализированы переменные:
    * **TEST_SIZE** для фиксирования размера тестовой выборки при разбиении наборов данных;
    * **SEED** для фиксирования случайности.
3. Произведена загрузка данных в рабочую среду Jupyter Notebook. Инициализирована переменная `datagen`. Набор данных состоит из 7_591 изображений;
4. Произведено разделение исходного набора на выборки:
    * `datagen_flow_train` - обучающая выборка;
    * `datagen_flow_test` - тестовая выборка.
5. Выведены на экран основные параметры набора данных `labels`:
    * набор данных состоит из 2 столбцов и 7591 строк;
    * в наборе отсутствуют пропущенные значения.
6. Выведены на экран основные параметры распределения данных по признаку `real_age`:
    * распределение имеет вид, приближенный к нормальному, с небольшим скосом вправо. Стоит отметить, что в наборе присутствуют так же слишком юные покупатели - дети;
    * присутствуют выбросы в виде аномально больших значений - их доля составляет 2.23% от всего набора;
    * медианное значение возраста покупателей - 29 лет;
    * средний возраст покупателей - 31.2 лет.

В целом, набор данных не требует дополнительных преобразований помимо тех, что уже были выполнены на этапе загрузки.
При построении модели нейронной сети и при дальнейшем получении прогнозов будем учитывать, что в наборе присутствуют аномальные значения. А значит, итоговые прогнозные значения могут быть выше, чем реальный возраст покупателей.

7. Построена модель нейронной сети для анализа изображений покупателей. Инициализированы пользовательские функции:
    * load_train - загрузка обучающего набора данных;
    * load_test - загрузка тестового набора данных;
    * create_model - создание модели нейронной сети;
    * train_model - обучение модели нейронной сети.
8. По результатам обучения модели на GPU-тренажере, удалось достичь целевого значения метрики `MAE` - 7.2710.

Модель неплохо справилась с задачей - поставленная цель была достигнута. Улучшить качество прогноза можно путем дополнительных преобразований набора данных и усложнения нейронной сети. Также глобально можно было бы дополнить набор данных большим количеством фотографий.